In [1]:
import csv
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions
import pandas as pd
import csv
import json
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pandas as pd
import time

In [2]:
test = []
names = []
flipPrices = []
prodNames = []
info,price = [],[]
url = "https://flipkart.com/search?q="
# query to search for.
q = input("Enter product name (Searching on flipkart)")
q = q.replace(" ","+")
#     q = input("Enter a query: ")
file_name = q.replace(" ", "_")
# response recieved in bytes
resp = requests.get(url+q)
# parsing response content using BeautifulSoup class, so that we can perform operations on it.
parsed_html = bs(resp.content, 'html.parser')
# data cleaning
raw_data = parsed_html.find("script", attrs={"id":"is_script"})
data = raw_data.contents[0].replace("window.__INITIAL_STATE__ = ","").replace(";","")
json_data = json.loads(data)
req_data = json_data["pageDataV4"]["page"]["data"]["10003"]   #[10]["widget"]["data"]["products"][3]["productInfo"]
#req_json_data = json_data["seoMeta"]["answerBox"]["data"]["renderableComponents"][0]["value"]["data"]

In [3]:
data_list = []
# print(len(req_data))
try:
    for i in range(1, len(req_data)):
        d = {}
        jd = req_data[i]["widget"]["data"]["products"]
        # print(len(jd))
        # print("i: ", i, end="\n")
        for j in range(len(jd)):
            jd2 = jd[j]["productInfo"]["value"]

            d["title"] = jd2["titles"]["title"]
            d["keySpecs"] = jd2["keySpecs"]
            d["rating"] = jd2["rating"]["average"]
            d["ratingCount"] = jd2["rating"]["count"]
            d["price"] = jd2["pricing"]["finalPrice"]["value"]
            #d["warranty"] = jd2["warrantySummary"]
            d["url"] = jd2["smartUrl"]

            # You can uncomment below lines if you want to print json output on terminal

            # print("Title: ",jd2["titles"]["title"],end="\n")
            # print("key specs: ", jd2["keySpecs"], end="\n")
            # print("Rating: ", jd2["rating"]["average"], end="\n")
            # print("Total ratings: ", jd2["rating"]["count"], end="\n")
            # print("Price: ", jd2["pricing"]["finalPrice"]["value"],end="\n")
            # print("warranty: ", jd2["warrantySummary"], end="\n")
            # print("Smart url: ", jd2["smartUrl"], end="\n")
        data_list.append(d)
        
except:
    pass 
# dumping data to result.json file
    # print(list(data_list))
with open("flipkart"+'.json', 'w') as fp:
    json.dump(data_list, fp)

# Now let us write our data to csv file
data_file = open("flipkart"+'.csv', 'w') 

# create the csv writer object 
csv_writer = csv.writer(data_file) 

# Counter variable used for writing  
# headers to the CSV file 
count = 0




In [4]:
for data in data_list:
    if count == 0: 

        # Writing headers of CSV file 
        header = data.keys() 
        csv_writer.writerow(header) 
        count += 1
        
    # Writing data of CSV file 

    csv_writer.writerow(data.values()) 


with open('flipkart.csv') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    rows = list(reader)
i,j = 0,2
while i in range(len(rows)):
    try:
        name = rows[j][0]
        #name = " ".join(name.split(' ')[0:2])
        #print(name)
        #print("name = ",name)
        names.append(name)
        i += 1
        j += 2
    except:
            break


In [5]:
print("Best results",len(names))
#print(names,len(names))
if len(names) <= 10:
    flipkart_url = "https://www.flipkart.com/search?q=" + q
    print(flipkart_url)
    uClient = uReq(flipkart_url)
    flipkartPage = uClient.read()
    uClient.close()
    flipkart_html = bs(flipkartPage, "html.parser")
    bigboxes = flipkart_html.find_all("a", {"class": "s1Q9rs"})
    soup = BeautifulSoup(flipkartPage, 'html.parser')
    info = soup.select("[class~=s1Q9rs]")
    flipPrices = soup.select("[class =_30jeq3]")
    prodNames = [i.get('title') for i in info]
    names = prodNames
    df = pd.DataFrame(list(zip(prodNames, flipPrices)), 
                columns =['product_name', 'Flipkart_price']) 
    df.to_csv('flipkart.csv')
    print(df)

else:
    with open('flipkart.csv') as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        rows = list(reader)

Best results 0
https://www.flipkart.com/search?q=4gb+RAM+Laptop
                                         product_name Flipkart_price
0   Lenovo IdeaPad Core i3 10th Gen - (4 GB/256 GB...      [₹31,490]
1   ASUS VivoBook 14 (2021) Celeron Dual Core - (4...      [₹25,990]
2   Lenovo IdeaPad 3 CB Celeron Dual Core - (4 GB/...      [₹39,990]
3   ASUS (2022) Celeron Dual Core - (4 GB/128 GB S...      [₹20,691]
4   Infinix INBook X1 Neo Series Celeron Quad Core...      [₹22,990]
5   SAMSUNG Galaxy Book Go Snapdragon 7c Gen 2 - (...      [₹32,990]
6   ASUS Core i3 11th Gen - (4 GB/256 GB SSD/Windo...      [₹31,690]
7   acer Aspire 3 Dual Core 3020e - (4 GB/256 GB S...      [₹24,990]
8   ASUS (2022) Celeron Dual Core - (4 GB/128 GB S...      [₹30,490]
9   Lenovo Core i3 11th Gen - (4 GB/256 GB SSD/Win...      [₹38,990]
10  DELL Vostro Core i3 10th Gen - (4 GB/1 TB HDD/...      [₹42,278]
11  ASUS Chromebook Celeron Dual Core - (4 GB/64 G...      [₹21,490]
12  Lenovo IdeaPad 3 Celeron Dual Core 

In [6]:
    i,j = 0,2
    while i in range(len(rows)):
        try:
            price = rows[j][4]
            #price = price[i].text
            #print("price = ",price)
            flipPrices.append(price)
            i += 1
            j += 2
        except:
            break
    df = pd.DataFrame(list(zip(names, flipPrices)), 
            columns =['Product_name', 'Flipkart_price'])

    print(df)
    df.to_csv('flipkart.csv')

data_file.close()

                                         Product_name Flipkart_price
0   Lenovo IdeaPad Core i3 10th Gen - (4 GB/256 GB...      [₹31,490]
1   ASUS VivoBook 14 (2021) Celeron Dual Core - (4...      [₹25,990]
2   Lenovo IdeaPad 3 CB Celeron Dual Core - (4 GB/...      [₹39,990]
3   ASUS (2022) Celeron Dual Core - (4 GB/128 GB S...      [₹20,691]
4   Infinix INBook X1 Neo Series Celeron Quad Core...      [₹22,990]
5   SAMSUNG Galaxy Book Go Snapdragon 7c Gen 2 - (...      [₹32,990]
6   ASUS Core i3 11th Gen - (4 GB/256 GB SSD/Windo...      [₹31,690]
7   acer Aspire 3 Dual Core 3020e - (4 GB/256 GB S...      [₹24,990]
8   ASUS (2022) Celeron Dual Core - (4 GB/128 GB S...      [₹30,490]
9   Lenovo Core i3 11th Gen - (4 GB/256 GB SSD/Win...      [₹38,990]
10  DELL Vostro Core i3 10th Gen - (4 GB/1 TB HDD/...      [₹42,278]
11  ASUS Chromebook Celeron Dual Core - (4 GB/64 G...      [₹21,490]
12  Lenovo IdeaPad 3 Celeron Dual Core 4th Gen - (...      [₹27,990]
13  ASUS Pentium Quad Core - (4 GB